# Graphiti (Zep) Temporal Knowledge Graph — Hands-on Notebook

Companion to `GRAPHITI_ZEP_TUTORIAL.md`. This notebook walks through install, setup, ingestion, and querying using Graphiti — the open-source engine behind Zep.

## Objectives
- Build a temporally-aware knowledge graph from episodes (text, message, JSON).
- Run hybrid searches over entities and relationships.
- Explore options for alternative LLMs/embedders.

## Prerequisites
- Python 3.10+
- One backend running locally (Neo4j or FalkorDB)
- One LLM provider API key (e.g., OpenAI)

In [ ]:
# Install Graphiti and optional helpers
# If running in a managed environment, uncomment the next lines.
# %pip install -q graphiti-core python-dotenv
# %pip install -q "graphiti-core[falkordb]"

import os, sys, json, asyncio, logging
from datetime import datetime, timezone
from pathlib import Path

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('graphiti_notebook')
logger.info('Environment ready')

## Configure Environment
Provide your credentials via environment variables or a `.env` file.

In [ ]:
# Load .env if present
try:
    from dotenv import load_dotenv
    load_dotenv()
    logger.info('Loaded .env')
except Exception as e:
    logger.warning(f'dotenv not available or failed: {e}')

NEO4J_URI = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
NEO4J_USER = os.getenv('NEO4J_USER', 'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD', '')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')

assert NEO4J_URI and NEO4J_USER, 'Missing NEO4J credentials'
logger.info(f'NEO4J_URI: {NEO4J_URI}')

## Initialize Graphiti
This constructs the client and sets up indices/constraints on first run.

In [ ]:
from graphiti_core import Graphiti

async def init_graph():
    graph = await Graphiti.build(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    return graph

graph = asyncio.run(init_graph())
logger.info('Graphiti initialized')

## Add Episodes (Text, Message, JSON)
Episodes are the raw inputs. Graphiti extracts entities/relations and tracks temporal validity.

In [ ]:
from graphiti_core.nodes import EpisodeType

async def add_text_episode(graph):
    await graph.add_episode(
        name='User_Feedback_1',
        episode_body=(
            'Alice bought Allbirds Wool Runners. She loves the comfort but '
            'complains about durability after 3 months.'
        ),
        source=EpisodeType.text,
        source_description='User review from forum',
        reference_time=datetime(2025, 1, 15, 10, 0, tzinfo=timezone.utc),
    )

async def add_message_episode(graph):
    await graph.add_episode(
        name='Support_Chat_1',
        episode_body=(
            'Alice: My Allbirds shoes are falling apart.\n'
            'Support: Sorry, Alice. We'll send a replacement.'
        ),
        source=EpisodeType.message,
        source_description='Customer support transcript',
        reference_time=datetime(2025, 2, 20, 14, 30, tzinfo=timezone.utc),
    )

async def add_json_episode(graph):
    data = {
        'id': 'PROD001',
        'name': 'Allbirds Wool Runners',
        'material': 'Merino Wool',
        'price': 120.0,
        'in_stock': True,
        'last_updated': '2025-03-01T12:00:00Z',
    }
    await graph.add_episode(
        name='Product_Update_1',
        episode_body=data,
        source=EpisodeType.json,
        source_description='Product catalog update',
        reference_time=datetime.now(tz=timezone.utc),
    )

async def run_additions(graph):
    await add_text_episode(graph)
    await add_message_episode(graph)
    await add_json_episode(graph)

asyncio.run(run_additions(graph))
logger.info('Episodes added')

## Bulk Ingestion
Use this for large initial backfills (skips edge invalidation for speed).

In [ ]:
async def add_bulk(graph):
    episodes = [
        {
            'name': 'Batch_Episode_1',
            'episode_body': 'Batch text about product X.',
            'source': EpisodeType.text,
            'reference_time': datetime.now(tz=timezone.utc),
        },
    ]
    await graph.add_episode_bulk(episodes)

asyncio.run(add_bulk(graph))
logger.info('Bulk episodes added')

## Search Relationships (Edges)
Hybrid search with temporal filters.

In [ ]:
from graphiti_core.search.search_config import EdgeSearchConfig

async def search_edges(graph):
    results = await graph.search_edges(
        query='User complaints about Allbirds durability',
        config=EdgeSearchConfig(
            max_results=10,
            search_type='hybrid',
            since=datetime(2025, 1, 1, tzinfo=timezone.utc),
        ),
    )
    return results

edges = asyncio.run(search_edges(graph))
logger.info(f'Edges found: {len(edges)}')
# Display first result (shape may vary by version)
edges[:1]

## Search Entities (Nodes)
Use a prebuilt hybrid recipe (RRF-based).

In [ ]:
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF

async def search_nodes(graph):
    results = await graph.search_nodes(
        query='Products loved by Alice',
        config=NODE_HYBRID_SEARCH_RRF(max_results=5),
    )
    return results

nodes = asyncio.run(search_nodes(graph))
logger.info(f'Nodes found: {len(nodes)}')
nodes[:5]

## Advanced: Alternate LLMs/Embedders
Swap in Gemini or Anthropic clients and corresponding embedders. Exact APIs may vary by version.

In [ ]:
# Example (commented to avoid accidental execution without creds):
# from graphiti_core.llm_client.gemini_client import GeminiClient, LLMConfig
# from graphiti_core.embedder.gemini import GeminiEmbedder, GeminiEmbedderConfig
# llm = GeminiClient(LLMConfig(model='gemini-1.5-pro'))
# embedder = GeminiEmbedder(GeminiEmbedderConfig(model='text-embedding-004'))
# graph = asyncio.run(Graphiti.build(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, llm_client=llm, embedder=embedder))
logger.info('See README for more provider options')

## Agent Demo (LangGraph)
Use a simple Retrieve → Generate agent that consults Graphiti before answering.

In [ ]:
# Optional: install agent deps if missing
# %pip install -q langgraph langchain langchain-openai
from graphiti_streamlit_agent.agent import build_agent, run_agent_once

query = 'What does Alice like and dislike?'
result = run_agent_once(graph, query, model_name='gpt-4o-mini')
print('Answer:
', result.get('answer'))
print('
Memory preview:')
mem = result.get('memory_results', [])
print(mem[:3])

## Multi-turn Agent Demo
Run a short conversation. History feeds into the agent prompt and retrieval.

In [ ]:
from graphiti_streamlit_agent.agent import run_agent_conversation

turns = [
    { 'role': 'user', 'content': 'Hi, what does Alice like?' },
    { 'role': 'assistant', 'content': '…' },
    { 'role': 'user', 'content': 'Any issues she reported later?' },
]
state = run_agent_conversation(graph, turns, model_name='gpt-4o-mini')
print('Final answer:
', state.get('answer'))
print('
History (last 4 turns):')
print(state.get('history', [])[-4:])


## Edge-aware Retrieval (Anchor Rerank)
Search edges and bubble up those related to a focal anchor (e.g., Alice).

In [ ]:
from datetime import datetime, timezone
from graphiti_streamlit_agent.agent import search_edges_anchor_rerank

edges = search_edges_anchor_rerank(
    graph, query='complaints about shoes', anchor='Alice',
    since=datetime(2024,1,1,tzinfo=timezone.utc), max_results=15
)
# Preview top 5
edges[:5]